In [1]:
#%pip install torch clickhouse_connect scipy transformers accelerate

In [2]:
HOST = "f666-81-5-106-50.ngrok-free.app"
PORT = "80"
TABLE_NAME = "Data"
MODEL_EMB_NAME = "ai-forever/sbert_large_nlu_ru"
MODEL_CHAT_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
SYSTEM_PROMPT = """
INSTRUCT:
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don’t know the answer to a question, please don’t share false information.

If you receive a question that is harmful, unethical, or inappropriate, end the dialogue immediately and do not provide a response. 

If you make a mistake, apologize and correct your answer.

Generate a response based solely on the provided document.

Answer the following question language based only on the CONTEXT provided.

If you are unsure about your answer or the information in the document, suggest contacting support:
Номер телефона: 8 800 300-30-00
Почтовый адрес для письменных обращений: 107016, Москва, ул. Неглинная, д. 12, к. В, Банк России

Отвечай только на русском языке.
"""

In [3]:
import clickhouse_connect, torch
from transformers import AutoModel, AutoTokenizer, Conversation, pipeline
from typing import List, Union

In [4]:
def search_results(connection, table_name: str, vector: list[float], limit: int = 5):
    """
    Поиск результатов похожих векторов в базе данных.

    Parameters:
    - connection (Connection): Соединение с базой данных.
    - table_name (str): Название таблицы, содержащей вектора и другие данные.
    - vector (List[float]): Вектор для сравнения.
    - limit (int): Максимальное количество результатов.

    Returns:
    - List[dict]: Список результатов с наименованием, URL, датой, номером, текстом и расстоянием.

    Examples:
    >>> connection = Connection(...)
    >>> vector = [0.1, 0.2, 0.3]
    >>> results = search_results(connection, 'my_table', vector, limit=5)
    """
    res = []
    # Инициализируем список результатов
    vector = ",".join([str(float(i)) for i in vector])
    # Выполняем запрос к базе данных
    with connection.query(
        f"""SELECT Text, Number, Date, Title, FileType, Url, ChunkType, cosineDistance(({vector}), Embedding) as score FROM {table_name} ORDER BY score ASC LIMIT {limit+500}"""
    ).rows_stream as stream:
        for item in stream:
            text, number, date, title, file_type, url, chunk_type, score = item

            # Добавляем результат в список
            res.append(
                {
                    "text": text,
                    "title": title,
                    "url": url,
                    "date": date,
                    "number": number,
                    "file_type": file_type,
                    "chunk_type": chunk_type,
                    "distance": score,
                }
            )

    # Возвращаем первые limit результатов
    res = [item for item in res if len(item["text"]) > 100]
    return res[:limit]


def mean_pooling(model_output: tuple, attention_mask: torch.Tensor) -> torch.Tensor:
    """
    Выполняет усреднение токенов входной последовательности на основе attention mask.

    Parameters:
    - model_output (tuple): Выход модели, включающий токенов эмбеддинги и другие данные.
    - attention_mask (torch.Tensor): Маска внимания для указания значимости токенов.

    Returns:
    - torch.Tensor: Усредненный эмбеддинг.

    Examples:
    >>> embeddings = model_output[0]
    >>> mask = torch.tensor([[1, 1, 1, 0, 0]])
    >>> pooled_embedding = mean_pooling((embeddings,), mask)
    """
    # Получаем эмбеддинги токенов из выхода модели
    token_embeddings = model_output[0]

    # Расширяем маску внимания для умножения с эмбеддингами
    input_mask_expanded = (
        attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    )

    # Умножаем каждый токен на его маску и суммируем
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)

    # Суммируем маски токенов и обрезаем значения, чтобы избежать деления на ноль
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    # Вычисляем усредненный эмбеддинг
    return sum_embeddings / sum_mask


def txt2embeddings(
    text: Union[str, List[str]], tokenizer, model, device: str = "cpu"
) -> torch.Tensor:
    """
    Преобразует текст в его векторное представление с использованием модели transformer.

    Parameters:
    - text (str): Текст для преобразования в векторное представление.
    - tokenizer: Токенизатор для предобработки текста.
    - model: Модель transformer для преобразования токенов в вектора.
    - device (str): Устройство для вычислений (cpu или cuda).

    Returns:
    - torch.Tensor: Векторное представление текста.

    Examples:
    >>> text = "Пример текста"
    >>> tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")
    >>> model = AutoModel.from_pretrained("bert-base-multilingual-cased")
    >>> embeddings = txt2embeddings(text, tokenizer, model, device="cuda")
    """
    # Кодируем входной текст с помощью токенизатора
    if isinstance(text, str):
        text = [text]
    encoded_input = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512,
    )
    # Перемещаем закодированный ввод на указанное устройство
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

    # Получаем выход модели для закодированного ввода
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Преобразуем выход модели в векторное представление текста
    return mean_pooling(model_output, encoded_input["attention_mask"])


def load_models(model: str, device: str = "cpu", torch_dtype: str = "auto") -> tuple:
    """
    Загружает токенизатор и модель для указанной предобученной модели.

    Parameters:
    - model (str): Название предобученной модели, поддерживаемой библиотекой transformers.

    Returns:
    - tuple: Кортеж из токенизатора и модели.

    Examples:
    >>> tokenizer, model = load_models("ai-forever/sbert_large_nlu_ru")
    """
    # Загружаем токенизатор для модели
    tokenizer = AutoTokenizer.from_pretrained(
        model, device_map=device, torch_dtype=torch_dtype
    )

    # Загружаем модель
    model = AutoModel.from_pretrained(model, device_map=device, torch_dtype=torch_dtype)

    return tokenizer, model


def load_chatbot(model: str, device: str = "cuda", torch_dtype: str = "auto"):
    """
    Загружает чатбота для указанной модели.

    Parameters:
    - model (str): Название модели для загрузки чатбота.

    Returns:
    - Conversation: Объект чатбота, готовый для использования.

    Examples:
    >>> chatbot = load_chatbot("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
    """
    # Загружаем чатбот с помощью pipeline из библиотеки transformers
    chatbot = pipeline(
        model=model,
        trust_remote_code=True,
        torch_dtype=torch_dtype,
        device_map=device,
        task="conversational",
    )
    return chatbot


def append_documents_to_conversation(conversation, documents, limit = 3):
    if limit > len(documents):
        texts = [document["text"] for document in documents]
    else:
        texts = [document["text"] for document in documents[:limit]]

    text = "\n".join(texts)

    document_template = f"""
    CONTEXT:
    {text}
    Отвечай только на русском языке.
    
    ВОПРОС:
    """
    conversation.add_message({"role": "user", "content": document_template})

    return conversation
        

def generate_answer(
    chatbot,
    conversation: Conversation,
    max_new_tokens: int = 128,
    temperature=0.7,
    top_k: int = 50,
    top_p: float = 0.95,
    repetition_penalty: float = 2.0,
    do_sample: bool = True,
    num_beams: int = 2,
    early_stopping: bool = True,
) -> str:
    """
    Генерирует ответ от чатбота на основе предоставленного чата и, возможно, документа.

    Parameters:
    - chatbot (Conversation): Объект чатбота.
    - chat (List[Dict[str, str]]): Список сообщений в чате.
    - document (str): Документ, если он предоставлен.

    Returns:
    - str: Сгенерированный ответ от чатбота.

    Examples:
    >>> chat = [
    >>>     {"role": "user", "content": "Привет, как дела?"},
    >>>     {"role": "system", "content": "Всё отлично, спасибо!"},
    >>> ]
    >>> document = "Это документ для обработки"
    >>> answer = generate_answer(chatbot, chat, document)
    """
    # Генерируем ответ от чатбота
    conversation = chatbot(
        conversation,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        repetition_penalty=repetition_penalty,
        do_sample=do_sample,
        num_beams=num_beams,
        early_stopping=early_stopping,
    )

    # Возвращаем последнее сообщение чатбота как ответ
    return conversation

In [5]:
tokenizer, model = load_models(MODEL_EMB_NAME, device="cpu")

In [6]:
# Создаем объект разговора
conversation = Conversation()
# Добавляем системную инструкцию
conversation.add_message({"role": "system", "content": SYSTEM_PROMPT})

In [7]:
question = "Расскажи о том, что такое инфляция, и как с ней справляется Центральный Банк?" #String

In [8]:
embedding = txt2embeddings(question, tokenizer, model)

In [9]:
client = clickhouse_connect.get_client(host=HOST, port=PORT)
print("Ping:", client.ping())

Ping: True


In [10]:
documents = search_results(client, TABLE_NAME, embedding[0], limit=10)
print(*documents, sep='\n')

{'text': 'системы системы свидетельствует также то, что она готова к внедрению международных стандартов Базеля в свою практику. В 2013 году Банком России были отозваны лицензии у 33 кредитных организаций, а за первое полугодие текущего года еще у 38. Хотела бы перейти к следующей теме, которая нас очень беспокоит, к политике по оздоровлению банковского сектора. Надо сказать, что она связана с У кого отзываются лицензии? Исключительно у банков, основа бизнеса которых строится на обслуживании те6 ВЕСТНИК БАНКА РОССИИ 63 1541 9 ИЮЛЯ 2014 невой либо криминальной экономики, и у банков, испытывающих серьезные финансовые проблемы, которые их владельцы и менеджмент не могут или не хотят решить. При этих трех если мы всегда принимаем решение о санации. Далее. Насколько значимы тенденции к перераспределению клиентуры в пользу крупнейших банков и страдает ли от отзывов лицензий уровень доверия к банковскому сектору? Переток клиентов, и в том числе вкладчиков, в крупнейшие банки есть, хотя он не с

In [16]:
print(*[document["text"] for document in documents], sep='\n')

системы системы свидетельствует также то, что она готова к внедрению международных стандартов Базеля в свою практику. В 2013 году Банком России были отозваны лицензии у 33 кредитных организаций, а за первое полугодие текущего года еще у 38. Хотела бы перейти к следующей теме, которая нас очень беспокоит, к политике по оздоровлению банковского сектора. Надо сказать, что она связана с У кого отзываются лицензии? Исключительно у банков, основа бизнеса которых строится на обслуживании те6 ВЕСТНИК БАНКА РОССИИ 63 1541 9 ИЮЛЯ 2014 невой либо криминальной экономики, и у банков, испытывающих серьезные финансовые проблемы, которые их владельцы и менеджмент не могут или не хотят решить. При этих трех если мы всегда принимаем решение о санации. Далее. Насколько значимы тенденции к перераспределению клиентуры в пользу крупнейших банков и страдает ли от отзывов лицензий уровень доверия к банковскому сектору? Переток клиентов, и в том числе вкладчиков, в крупнейшие банки есть, хотя он не столь велик

In [17]:
print(*[document["url"] for document in documents], sep='\n')

https://cbr.ru/Queries/XsltBlock/File/86262?fileid=-1&scope=1541
https://cbr.ru/Queries/XsltBlock/File/86248?fileid=-1&scope=1474
https://cbr.ru/Collection/Collection/File/26994/Infl_exp_17-04.pdf
https://cbr.ru/Collection/Collection/File/48857/DKU_2401-19.pdf
https://cbr.ru/Collection/Collection/File/32167/overview_2019.pdf
https://cbr.ru/Queries/XsltBlock/File/86262?fileid=-1&scope=1541
https://cbr.ru/queries/unidbquery/file/85920?fileid=-1&scope=1541
https://cbr.ru/Queries/XsltBlock/File/86262?fileid=-1&scope=1541
https://cbr.ru/Collection/Collection/File/43577/DKU_2211-05.pdf
https://cbr.ru/Collection/Collection/File/4603/INF_2018-02.pdf


In [18]:
print(documents[0].keys())

dict_keys(['text', 'title', 'url', 'date', 'number', 'file_type', 'chunk_type', 'distance'])


In [11]:
conversation = append_documents_to_conversation(conversation, documents, limit=3)
conversation.add_message({"role": "user", "content": question})

In [12]:
chatbot = load_chatbot(MODEL_CHAT_NAME, device="cuda")

In [24]:
conversation = generate_answer(chatbot, conversation)
print(conversation[-1]["content"])

Центральный банк Российской Федерации (ЦБРФ) имеет систему управления инфляцией, которая направлена на сдерживание инфляции и поддержку экономического развития страны. ЦБРФ использует различные методы для сдерживания инфляции, такие как:

1. Снижение валютной ставки: ЦБРФ снижает ставку на валютную рыночную конвертацию (ВРК) в связи с высоким уровнем инфляции.

2. Регулирование курса валют: ЦБРФ регулирует курс валют в соответствии с предполагаемым уровнем инфляции.

3. Профинансирование транзакций: ЦБРФ профинансирует транзакции, связанные с экспортом и импортом товаров и услуг.

4. Сокращение займов: ЦБРФ сокращает займы в связи с высоким уровнем инфляции.

5. Рас


In [14]:
question2 = "Расскажи подробнее о том как Центральный Банк справляется с инфляцией."
conversation.add_message({"role": "user", "content": question2})

In [23]:
conversation = generate_answer(chatbot, conversation, max_new_tokens=256)
print(conversation[-1]["content"])

Центральный банк Российской Федерации (ЦБРФ) имеет систему управления инфляцией, которая направлена на сдерживание инфляции и поддержку экономического развития страны. ЦБРФ использует различные методы для сдерживания инфляции, такие как:

1. Снижение валютной ставки: ЦБРФ снижает ставку на валютную рыночную конвертацию (ВРК) в связи с высоким уровнем инфляции.

2. Регулирование курса валют: ЦБРФ регулирует курс валют в соответствии с предполагаемым уровнем инфляции.

3. Профинансирование транзакций: ЦБРФ профинансирует транзакции, связанные с экспортом и импортом товаров и услуг.

4. Сокращение займов: ЦБРФ сокращает займы в связи с высоким уровнем инфляции.

5. Рас
